In [2]:
!pip install pyspark==3.2.1

    100% |################################| 281.4MB 4.3kB/s eta 0:00:01
    100% |################################| 204kB 5.7MB/s eta 0:00:01
  Running setup.py bdist_wheel for pyspark ... done
  Stored in directory: /root/.cache/pip/wheels/23/35/61/f720c76ff25c680383cd010cd524fa66c91c9b62a4b0aebe9f
Successfully built pyspark


In [4]:
!pip install findspark

In [5]:
# import libraries 
import socket
import sys
import os
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SparkSession
import findspark
import time 

In [21]:
# 전역변수 설정 
# local mode 
SPARK_LOCAL_MASTER = "local[3]"
# client mode 
SPARK_CLUSTER_MASTER = "yarn" #"spark://35.202.237.174:7077" 
SPARK_APP_NAME = "DATA-Preparation"
HOST_NAME = "spark-master" #socket.gethostname()


# Define path
DATA_ROOT = f'{os.getcwd()}/data'
DATA_PATH = f'{DATA_ROOT}'

In [7]:
print(DATA_PATH)
print(HOST_NAME)

/notebooks/data
spark-master


In [8]:
findspark.init()
print(os.getenv('SPARK_HOME'))

/usr/local/lib/python3.6/dist-packages/pyspark


In [9]:
print(f"JAVA_HOME : {os.getenv('JAVA_HOME')}")
print(f"PATH : {os.getenv('PATH')}")
os.getcwd()

JAVA_HOME : None
PATH : /usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin


'/notebooks'

In [22]:
# Spark session 생성 메서드 
def init_remote_session():
    #SPARK_CLUSTER_MASTER = "spark://34.64.108.172:7077" 
    spark = SparkSession.builder.master(SPARK_CLUSTER_MASTER).appName(SPARK_APP_NAME).getOrCreate()
    return spark
    
# local mode 실행 시 메모리를 확장해야 하는 경우 있어서, conf에 memory 변경 추가  
def init_local_session():
    #SPARK_LOCAL_MASTER = "local[3]"
    spark = SparkSession.builder.master(SPARK_LOCAL_MASTER).appName(SPARK_APP_NAME).config('spark.driver.host', HOST_NAME).getOrCreate()
    default_conf = spark.sparkContext._conf#.getAll()
    print(f'Old Conf : {default_conf.getAll()}')
    conf = spark.sparkContext._conf.setAll([
        ('spark.executor.instances', 1)
        #, ('spark.driver.memory', '12g'), ('spark.executor.memory', '8g'), ('spark.driver.maxResultSize', '8g')
        , ('spark.driver.allowMultipleContexts', 'true'), ('spark.sql.shuffle.partitions', 8)
        ##,('spark.memory.offHeap.enabled', True), ('spark.memory.offHeap.size', '8g')
    ])
    spark.sparkContext.stop()
    
    spark = SparkSession.builder.master(SPARK_LOCAL_MASTER).appName(SPARK_APP_NAME).config(conf=default_conf).getOrCreate()
    new_conf = spark.sparkContext._conf
    print(f'Updated Conf : {new_conf.getAll()}')
    return spark

In [ ]:
# 스파크 생성 
#spark = init_local_session()
# 스파크 클러스터를 실행해 두었다고 가정
spark = init_remote_session()
sc = spark.sparkContext
sc

In [19]:
spark.stop()